# Imports

Clone and install https://github.com/MTG/pycompmusic

Download data using `download-dunya.py`

In [ ]:
import json, os, sys
import pickle, csv
import time
import datetime
from collections import OrderedDict
import itertools
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px

import scipy
import librosa
from IPython.display import Audio as ipy_audio

In [ ]:
from quicktranscribe import tonic, pitch, wave

# Data

In [ ]:
data_dir = "/Users/neerajaabhyankar/Repos/icm-shruti-analysis/data-dunya-hindustani/"
track = "Omkar Dadarkar - Raag Bhoopali"

Metadata, Tonic, Pitch, Audio

In [ ]:
# metadata = tonic.read_metadata(data_dir + track + ".json")
ctonic = tonic.read_tonic(data_dir + track + ".ctonic.txt")
pitch_annotations, aps = pitch.read_pitch(data_dir + track + ".pitch.txt")

In [ ]:
y, sr = wave.get_audio(data_dir + track + ".mp3")

# Validating Pitch Annotations

In [ ]:
pv = pitch.PitchValidator(audio_array=y, sampling_rate=sr)
pv.set_annotation(pitch_annotations=pitch_annotations, annotation_rate=aps)

In [ ]:
# Find a representative sample
# 06:45 to 07:00
pv.play_sample(start_time=6*60+45, end_time=7*60)

In [ ]:
pv.validate_annotations(start_time=6*60+45, end_time=7*60)

# Manual Note Mode Finding

In [ ]:
pv.plot_annotations_hist()

Add counts to the list manual-shruti-peaks.txt

# TODO

NEXT:
1. instead of using the annotations, auto-detect within sub_beats
2. think about tempo chunking
2. modularize and do this for a longer period of time

In [ ]:
sub_beat_factor = 2

Select a sample

In [ ]:
# 06:45 to 07:00
start = 45*60+20
end = 45*60+32

y_small = y[start*sr:end*sr]
pa_small = pitch_annotations[start*int(aps):end*int(aps), 1]

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=y_small, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

In [ ]:
# dilate beat_times by a factor of sub_beat_factor (=2 => swar at every 1/2 beat)
sub_beat_times = np.empty(len(beat_times) * sub_beat_factor)
sub_beat_times[0::2] = np.insert((beat_times[:-1] + beat_times[1:]) / sub_beat_factor, 0, beat_times[0] / sub_beat_factor)
sub_beat_times[1::2] = beat_times


Collapse pitches to known swars<br>
(Temp: based on manually found peaks)

In [ ]:
swar = OrderedDict({
    "D.": 0.8362, "S": 0.997, "R": 1.11896, "G": 1.2565, "P": 1.501
})
swars = np.array(list(swar.values()))*ctonic
swars = np.concatenate((swars/2, swars, swars*2))

In [ ]:
swar_small = []
for pa in pa_small:
    if pa == 0:
        swar_small.append(0)
    else:
        idx = (np.abs(swars - pa)).argmin()
        swar_small.append(swars[idx])

In [ ]:
plt.plot(range(len(pa_small)), pa_small)
plt.plot(range(len(swar_small)), swar_small)

Look at how the mode sounds per sub_beat

In [ ]:
beat_swars = []
y_swar_small = []
for beat_idx in range(1, len(sub_beat_times)):
    beat_start = sub_beat_times[beat_idx-1]
    beat_end = sub_beat_times[beat_idx]
    swars_beat = swar_small[int(beat_start*aps):int(beat_end*aps)]
    
    idx = (np.abs(swars - scipy.stats.mode(swars_beat).mode)).argmin()
    beat_swars.append(swars[idx])
    
    tone = librosa.tone(swars[idx], sr=sr, length=(beat_end-beat_start)*sr)
    y_swar_small += tone.tolist()

    

In [ ]:
print("beat-wise swar-collapsed waveform")
ipy_audio(data=y_swar_small, rate=sr)

In [ ]:
print("original sample")
ipy_audio(data=y_small, rate=sr)

Not quite there!

Time series in φ

In [ ]:
swar_indices_small = [np.where(np.abs(swars-ss)<1e-3)[0] for ss in swar_small]
swar_indices_small = [si[0] if len(si) > 0 else 0 for si in swar_indices_small]

In [ ]:
symbols = [sw[0] for sw in swar.keys()]
symbol_indices_small = np.array(swar_indices_small)%len(symbols)
symbol_list = np.array([symbols[ii] for ii in symbol_indices_small])

In [ ]:
ss = "".join(symbol_list)
deduped_ss = ''.join(ch for i, ch in enumerate(ss) if i == 0 or ch != ss[i-1])

In [ ]:
print(ss)
print(deduped_ss)

In [ ]:
plt.plot(range(len(symbol_indices_small)), symbol_indices_small)

In [ ]:
bigrams = np.empty((len(symbols), len(symbols)), dtype=int)

In [ ]:
for ii, jj in itertools.product(range(len(symbols)), range(len(symbols))):
    bigrams[ii, jj] = ss.count(symbols[ii]+symbols[jj])

In [ ]:
bigrams

In [ ]:
mod_bigrams = deepcopy(bigrams)

In [ ]:
for ii in range(len(symbols)):
    mod_bigrams[ii, ii] = 0

In [ ]:
plt.imshow(mod_bigrams)